# Building a Conversational Agent with Context Awareness

Xây dựng Agent đàm thoại có nhận thức ngữ cảnh

## Tổng quan
Hướng dẫn này phác thảo quy trình tạo ra một agent đàm thoại có thể duy trì ngữ cảnh qua nhiều tương tác. Chúng ta sẽ sử dụng một framework AI hiện đại để xây dựng một agent có khả năng tham gia vào các cuộc trò chuyện tự nhiên và mạch lạc hơn.

## Động lực
Nhiều chatbot đơn giản thiếu khả năng duy trì ngữ cảnh, dẫn đến trải nghiệm người dùng rời rạc và khó chịu. Hướng dẫn này nhằm mục đích giải quyết vấn đề đó bằng cách triển khai một agent đàm thoại có thể ghi nhớ và tham khảo các phần trước của cuộc trò chuyện, nâng cao chất lượng tương tác tổng thể.

## Các thành phần chính
1.  **Language Model (Mô hình ngôn ngữ)**: Thành phần AI cốt lõi tạo ra các phản hồi.
2.  **Prompt Template (Mẫu Prompt)**: Định nghĩa cấu trúc của các cuộc trò chuyện của chúng ta.
3.  **History Manager (Trình quản lý lịch sử)**: Quản lý lịch sử và ngữ cảnh cuộc trò chuyện.
4.  **Message Store (Kho lưu trữ tin nhắn)**: Lưu trữ các tin nhắn cho mỗi phiên trò chuyện.

## Chi tiết phương pháp

### Thiết lập môi trường
Bắt đầu bằng cách thiết lập framework AI cần thiết và đảm bảo quyền truy cập vào một language model phù hợp. Điều này tạo thành nền tảng của agent đàm thoại của chúng ta.

### Tạo kho lưu trữ lịch sử trò chuyện
Triển khai một hệ thống để quản lý nhiều phiên trò chuyện. Mỗi phiên phải được xác định duy nhất và liên kết với lịch sử tin nhắn riêng của nó.

### Định nghĩa cấu trúc cuộc trò chuyện
Tạo một template bao gồm:
-   Một system message (tin nhắn hệ thống) định nghĩa vai trò của AI.
-   Một placeholder (vị trí giữ chỗ) cho lịch sử cuộc trò chuyện.
-   Đầu vào của người dùng.

Cấu trúc này hướng dẫn các phản hồi của AI và duy trì tính nhất quán trong suốt cuộc trò chuyện.

### Xây dựng chuỗi đàm thoại
Kết hợp prompt template với language model để tạo một chuỗi đàm thoại cơ bản. Bọc chuỗi này bằng một component quản lý lịch sử, tự động xử lý việc chèn và truy xuất lịch sử cuộc trò chuyện.

### Tương tác với Agent
Để sử dụng agent, hãy gọi nó với đầu vào của người dùng và một session identifier (định danh phiên). History manager đảm nhận việc truy xuất lịch sử cuộc trò chuyện thích hợp, chèn nó vào prompt và lưu trữ các tin nhắn mới sau mỗi tương tác.

## Kết luận
Phương pháp tạo agent đàm thoại này mang lại một số lợi thế:
-   **Context Awareness (Nhận thức ngữ cảnh)**: Agent có thể tham khảo các phần trước của cuộc trò chuyện, dẫn đến các tương tác tự nhiên hơn.
-   **Simplicity (Sự đơn giản)**: Thiết kế mô-đun giữ cho việc triển khai đơn giản.
-   **Flexibility (Tính linh hoạt)**: Dễ dàng sửa đổi cấu trúc cuộc trò chuyện hoặc chuyển sang một language model khác.
-   **Scalability (Khả năng mở rộng)**: Phương pháp dựa trên phiên cho phép quản lý nhiều cuộc trò chuyện độc lập.

Với nền tảng này, bạn có thể nâng cao hơn nữa agent bằng cách:
-   Triển khai prompt engineering (kỹ thuật prompt) phức tạp hơn.
-   Tích hợp nó với các knowledge base (cơ sở tri thức) bên ngoài.
-   Thêm các khả năng chuyên biệt cho các domain (lĩnh vực) cụ thể.
-   Kết hợp xử lý lỗi và các chiến lược sửa chữa cuộc trò chuyện.

Bằng cách tập trung vào quản lý ngữ cảnh, thiết kế agent đàm thoại này cải thiện đáng kể chức năng chatbot cơ bản, mở đường cho các trợ lý AI hấp dẫn và hữu ích hơn.

- Requirements

```bash
pip install -q langchain langchain_experimental openai python-dotenv langchain_ollama
```

In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOllama(model='qwen2.5:7b', temperature=0, num_predict=1000)

###  Create a simple in-memory store for chat histories

In [2]:
store = {}

def get_chat_history(session_id: str):
  if session_id not in store:
    store[session_id] = ChatMessageHistory()
  return store[session_id]

In [3]:
# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful AI assistant"),
  MessagesPlaceholder(variable_name="history"),
  ("human", "{input}")
])


In [4]:
# Combine the prompt and the model into a runable chain
chain = prompt | llm 

In [5]:
# Wrap the chain with message history
chain_with_history = RunnableWithMessageHistory(
  chain,
  get_chat_history,
  input_messages_key="input",
  history_messages_key='history'
)

In [6]:
session_id="userA"

response = chain_with_history.invoke(
  {"input": "Tên tôi là Dino"},
  config={"configurable": {"session_id": session_id}}
)
print("AI: ", response.content)

AI:  Rất vui được gặp bạn, Dino! Bạn có cần giúp đỡ gì hôm nay không? Tôi có thể hỗ trợ bạn với nhiều chủ đề khác nhau như tìm thông tin, giải đáp thắc mắc hoặc chỉ đơn giản là trò chuyện.


In [7]:
response2 = chain_with_history.invoke(
  {"input": "Tôi tên là gì?"},
  config={"configurable": {"session_id": session_id}}
)
print("AI: ", response2.content)

AI:  Bạn tên là Dino.


In [10]:
store[session_id].messages

[HumanMessage(content='Tên tôi là Dino', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Rất vui được gặp bạn, Dino! Bạn có cần giúp đỡ gì hôm nay không? Tôi có thể hỗ trợ bạn với nhiều chủ đề khác nhau như tìm thông tin, giải đáp thắc mắc hoặc chỉ đơn giản là trò chuyện.', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-03-20T09:35:56.453670741Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11817039901, 'load_duration': 2753618353, 'prompt_eval_count': 24, 'prompt_eval_duration': 1663000000, 'eval_count': 54, 'eval_duration': 7394000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-37bd517c-ae56-48f3-8abd-d6413230ef20-0', usage_metadata={'input_tokens': 24, 'output_tokens': 54, 'total_tokens': 78}),
 HumanMessage(content='Tôi tên là gì?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Bạn tên là Dino.', additional_kwargs={}, response_metadata={'model': 'qwen2.5

In [11]:
# Print the conversation history
for message in store[session_id].messages:
  print(f"{message.type}: {message.content}")

human: Tên tôi là Dino
ai: Rất vui được gặp bạn, Dino! Bạn có cần giúp đỡ gì hôm nay không? Tôi có thể hỗ trợ bạn với nhiều chủ đề khác nhau như tìm thông tin, giải đáp thắc mắc hoặc chỉ đơn giản là trò chuyện.
human: Tôi tên là gì?
ai: Bạn tên là Dino.
